In [1]:
from __future__ import division
from __future__ import absolute_import
from __future__ import print_function

import tensorflow as tf
import numpy as np

import tensorflow.examples.tutorials.mnist.input_data as input_data

tf.set_random_seed(2019)
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
train_set = mnist.train
test_set = mnist.test

In [4]:
import matplotlib.pyplot as plt

%matplotlib inline

fig, axes = plt.subplots(ncols=6, nrows=2)
plt.tight_layout(w_pad=-2.0, h_pad = -8.0)
images, labels = train_set.next_batch(12, shuffle=False)
for ind,(image,label) in enumerate(zip(images, labels)):
    image = image.reshape((28,28))
    label = label.argmax()
    row = ind//6
    col = ind % 6
    axes[row][col].imshow(image,cmap='gray')
    axes[row][col].axis('off')
    axes[row][col].set_title('%d' % label)

定义深度网络结构

In [ ]:
def hidden_layer(layer_input, output_depth, scope = "hidden_layer", reuse=None):
    input_depth = layer_input.get_shape()[-1]
    with tf.variable_scope(scope, reuse=reuse):
        w = tf.get_variable(initializer=tf.truncated_normal_initializer(stddev=0.1), 
                            shape=(input_depth, output_depth),name="weights")
        b = tf.get_variable(initializer=tf.constant_initializer(0.1), shape=(output_depth), name="bias")
        net = tf.matmul(layer_input,w)+b
        
        return net

def DNN(x, output_depths, scope='DNN', reuse=None):
    net=x
    for i , output_depth in enumerate(output_depths):
        net = hidden_layer(net,output_depth,scope='layer%d' % i, reuse=reuse)
        net = tf.nn.relu(net)
    net = hidden_layer(net,10, scope='classification', reuse=reuse)
    
    return net

In [6]:
input_ph = tf.placeholder(shape=(None,784), dtype=tf.float32)
label_ph = tf.placeholder(shape=(None,10), dtype=tf.int64)
#4层
dnn = DNN(input_ph, [400, 200, 100])

loss = tf.losses.softmax_cross_entropy(logits=dnn, onehot_labels=label_ph)
acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(dnn, axis=-1), tf.argmax(label_ph, axis=-1)),
                             dtype=tf.float32))
lr = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

In [13]:
batch_size = 64
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

for e in range(20000):
    images, labels = train_set.next_batch(batch_size)
    sess.run(train_op, feed_dict={input_ph: images, label_ph: labels})
    if e%1000 == 999:
        test_imgs, test_labels = test_set.next_batch(batch_size)
        loss_train, acc_train = sess.run([loss,acc], feed_dict={input_ph:images, label_ph: labels})
        loss_test, acc_test = sess.run([loss,acc], feed_dict={input_ph: test_imgs, label_ph: test_labels})
        print('STEP{}: train_loss: {:.6f} train_acc: {:.6f}  test_loss: {:.6f} test_acc: {:.6f}'.format(e+1, loss_train,acc_train,loss_test,acc_test))

print("Train Done !")
print('-'*30)

/usr/local/lib/python3.5/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


STEP1000: train_loss: 0.253653 train_acc: 0.921875  test_loss: 0.336481 test_acc: 0.906250
STEP2000: train_loss: 0.248945 train_acc: 0.937500  test_loss: 0.306756 test_acc: 0.921875
STEP3000: train_loss: 0.103879 train_acc: 1.000000  test_loss: 0.073909 test_acc: 1.000000
STEP4000: train_loss: 0.120947 train_acc: 0.937500  test_loss: 0.258381 test_acc: 0.890625
STEP5000: train_loss: 0.161666 train_acc: 0.953125  test_loss: 0.301629 test_acc: 0.875000
STEP6000: train_loss: 0.104920 train_acc: 0.968750  test_loss: 0.063372 test_acc: 0.984375
STEP7000: train_loss: 0.110567 train_acc: 0.984375  test_loss: 0.092887 test_acc: 0.968750
STEP8000: train_loss: 0.072268 train_acc: 1.000000  test_loss: 0.074406 test_acc: 0.984375
STEP9000: train_loss: 0.166160 train_acc: 0.937500  test_loss: 0.084829 test_acc: 0.968750
STEP10000: train_loss: 0.195742 train_acc: 0.937500  test_loss: 0.104386 test_acc: 0.937500
STEP11000: train_loss: 0.034972 train_acc: 1.000000  test_loss: 0.193936 test_acc: 0.9375

In [14]:
train_loss = []
train_acc = []
for _ in range(train_set.num_examples // 100):
    image, label = train_set.next_batch(100)
    loss_train, acc_train = sess.run([loss,acc], feed_dict={input_ph: image, label_ph: label})
    train_loss.append(loss_train)
    train_acc.append(acc_train)
print("Train loss: {:.6f}".format(np.array(train_loss).mean()))
print("Train accuracy: {:.6f}".format(np.array(train_acc).mean()))

Train loss: 0.054127
Train accuracy: 0.984582


In [17]:
test_loss = []
test_acc = []
for _ in range(test_set.num_examples // 100):
    image, label = test_set.next_batch(100)
    loss_test, acc_test = sess.run([loss, acc], feed_dict={input_ph: image, label_ph: label})
    test_loss.append(loss_test)
    test_acc.append(acc_test)
    
print("Test loss: {:.6f}".format(np.array(test_loss).mean()))
print("Test accuracy: {:.6f}".format(np.array(test_acc).mean()))

Test loss: 0.091533
Test accuracy: 0.971100


In [18]:
sess.close()

In [20]:
#loss_sum = tf.summary.scalar('loss',loss)
#w_hist = tf.summary.histogram('w_hist',w)
#iamge_sum = tf.summary.image('iamge', image)
#audio_sum = tf.summary.audio('audio', audio)

In [23]:
tf.reset_default_graph()

In [27]:
input_ph = tf.placeholder(shape=(None,784), dtype=tf.float32)
label_ph = tf.placeholder(shape=(None,10),dtype=tf.int64)

In [28]:
def weight_variable(shape):
    init = tf.truncated_normal(shape=shape, stddev=0.1)
    return tf.Variable(init)
def bias_variable(shape):
    init = tf.constant(0.1, shape=shape)
    return tf.Variable(init)

In [29]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
    with tf.name_scope('stddev'):
        stddev= tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))

In [30]:
def hidden_layer(x, output_dim, scope = 'hidden_layer', act = tf.nn.relu, reuse=None):
    input_dim = x.get_shape()[-1]
    with tf.name_scope(scope):
        with tf.name_scope('weight'):
            weight = weight_variable(input_dim, output_dim)
            variable_summaries(weight)
        with tf.name_scope('bias'):
            bias = bias_variable(output_dim)
            variable_summaries(bias)
        with name_scope('linear'):
            preact = tf.matmul(x,weight)+bias
            tf.summary.histogram('pre_activation',preact)
            
        output = act(preact)
        tf.summary.histogram('output', output)
        
        return output